In [1]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import os
import datetime

#Plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

# sklearn stuff
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, mean_squared_error, precision_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, Imputer, PolynomialFeatures
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from scipy.stats import mode,rankdata

import feature_pipelines as pipes

### Submission Functions 

In [2]:
def generate_regression_preds(reg, properties, model_name='pred_logerror', trx_date = '2016-12-01'):
    reg_preds = None
    # change month of properties
#     properties['month'] = month
#     properties['year'] = 2016
    properties['transactiondate'] = pd.Timestamp(trx_date)
    properties = add_date_features(properties)
    for i in range(int(properties.shape[0] / 100000)):   
        # get current test features
        current_test_feats = feature_pipeline.transform(properties.iloc[i*100000:(i+1)*100000])

        # predict on current test obs
        current_preds = Series(reg.predict(current_test_feats), name=model_name,
                              index = np.arange(i*100000,(i+1)*100000))

        if reg_preds is not None:
            reg_preds = pd.concat([reg_preds, current_preds])
        else:
            reg_preds = current_preds

    #  fencepost problem
    current_test_feats = feature_pipeline.transform(properties.iloc[2900000:])
    current_preds = Series(reg.predict(current_test_feats), name=model_name,
                          index = np.arange(2900000,2985217))
    reg_preds = pd.concat([reg_preds, current_preds])
    
    del properties['month']
    return reg_preds

In [3]:
def generate_submissions(oct_model,nov_model,dec_model,name='new_submission',logy=True):
    """
    This function creates the submission file for the public leaderboard predictions.
    Three already fitted models, one for each of the predicting time points, is required.
    """
    submission_df = DataFrame()
    for i in range(int(properties.shape[0] / 100000)):
        all_feats = full_pipeline.transform(properties.iloc[i*100000:(i+1)*100000])
        foo = properties.iloc[i*100000:(i+1)*100000][['parcelid']].reset_index(drop=True)
        if logy:
            foo = pd.concat([foo, DataFrame({'201610': oct_model.predict(all_feats),
                                                            '201611': nov_model.predict(all_feats),
                                                            '201612': dec_model.predict(all_feats)})], axis=1)
        else:
            foo = pd.concat([foo, DataFrame({'201610': np.log(oct_model.predict(all_feats)),
                                                            '201611': np.log(nov_model.predict(all_feats)),
                                                            '201612': np.log(dec_model.predict(all_feats))})], axis=1)
        submission_df = pd.concat([submission_df, foo], ignore_index=True)

    #  fencepost problem
    all_feats = full_pipeline.transform(properties.iloc[2900000:])
    foo = properties.iloc[2900000:][['parcelid']].reset_index(drop=True)
    foo = pd.concat([foo, DataFrame({'201610': oct_model.predict(all_feats),
                                                    '201611': nov_model.predict(all_feats),
                                                    '201612': dec_model.predict(all_feats)})], axis=1)
    submission_df = pd.concat([submission_df, foo], ignore_index=True)
    
    submission_df['201710'] = 0
    submission_df['201711'] = 0
    submission_df['201712'] = 0
    
    submission_df.rename(columns={'parcelid':'ParcelId'}, inplace=True)    
#     submission_df[['201610','201611','201612','201710','201711','201712']]= submission_df[['201610','201611','201612',
#                                                                                            '201710','201711','201712']].round(4)
    # unit test
    submission_df.drop_duplicates(inplace=True)
    assert submission_df.shape[0] == properties.shape[0]
    # write to .csv
    submission_df[['ParcelId','201610','201611','201612',
                  '201710','201711','201712']].to_csv(name + ".gz", index=False, float_format='%.4g', compression='gzip')
    return submission_df

In [4]:
def mean_absolute_errors(submission_df, comparison_df):
    """
    This function takes a submission entry for public leaderboard, and returns
    the training error for each month.
    """
    # training error
    trainresults = pd.merge(submission_df[['ParcelId','201610','201611','201612']], comparison_df[['parcelid','logerror','month']],
                           left_on='ParcelId', right_on='parcelid')
    oct_error = abs(trainresults[trainresults['month'] == 10]['201610'] 
                    - trainresults[trainresults['month'] == 10]['logerror']).mean()
    nov_error = abs(trainresults[trainresults['month'] == 11]['201611'] 
                    - trainresults[trainresults['month'] == 11]['logerror']).mean()
    dec_error = abs(trainresults[trainresults['month'] == 12]['201612'] 
                    - trainresults[trainresults['month'] == 12]['logerror']).mean()
    overall_mae = (oct_error*(trainresults['month'] == 10).sum() + nov_error*(trainresults['month'] == 11).sum() 
                        + dec_error*(trainresults['month'] == 12).sum()) / (trainresults['month'].isin([10,11,12])).sum()
    return (oct_error, nov_error, dec_error, overall_mae)

### Reading in data 

In [5]:
# similar to the1owl
def add_date_features(df):
    df["year"] = df["transactiondate"].dt.year
    df["month"] = df["transactiondate"].dt.month
    df["day"] = df["transactiondate"].dt.day
    df["quarter"] = df["transactiondate"].dt.quarter
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df

In [6]:
data = pd.read_csv("/home/anerdi/Desktop/Zillow/data/traindata20162017_addfeatures.csv.gz", compression='gzip',
                  parse_dates=['transactiondate'])

In [7]:
data = add_date_features(data)

### Data  Preprocessing Pipeline

In [47]:
# Setup variables considered in the model

# numerical variables
num_atts = ['garagetotalsqft', 
            'calculatedbathnbr',
            'structuretaxvaluedollarcnt',
            'bedroomcnt',
            'age']

num_atts_to_interact = ['calculatedfinishedsquarefeet', 'lotsizesquarefeet']

# categorical varaibles
cat_atts = ['airconditioningtypeid',
            'heatingorsystemtypeid',
            'Pool',
            'propertylandusetypeid',
            'taxdelinquencyflag',
            'architecturalstyletypeid',
            'regionidcounty',
            'month',
           'year']

# Dictionary of categorical variables and their default levels
cat_dict = {key:value for key,value in {'airconditioningtypeid':[-1] + list(range(1,14)),
           'architecturalstyletypeid':[-1] + list(range(1,28)),
           'buildingclasstypeid':[-1] + list(range(1,6)),
            'heatingorsystemtypeid':[-1] + list(range(1,26)),
            'pooltypeid10': list(range(-1,2)),
            'pooltypeid2': list(range(-1,2)),
            'pooltypeid7': list(range(-1,2)),
            'Pool': [0,1],
            'propertylandusetypeid': [-1, 31,46,47,246,247,248,260,261,262,263,264,265,266,267,268,269,270,271,
                                     273,274,275,276,279,290,291],
            'regionidcounty': [-1]+ [2061,3101,1286],
            'month': [-1] + list(range(1,13)),
            'year': [2016,2017],
            'quarter': [1,2,3,4],
            'zestimate_type': [1,2,3],
            'storytypeid':[-1] + list(range(1,36)),
            'typeconstructiontypeid':[-1] + list(range(1,19)),
            'yearbuilt': [-1] + list(range(1885,2018)),
            'fireplaceflag': [-1] + ['True','False'],
            'taxdelinquencyflag': [-1] + ['Y','N']
           }.items() if key in cat_atts}

# pairs to interact (x1,x2) where x1 is categorical and x2 is continuous
interact_pairs = [
                ('regionidcounty','calculatedbathnbr'),
                ('regionidcounty','bedroomcnt'),
                ('regionidcounty','structuretaxvaluedollarcnt'),
                ('regionidcounty','age'),
                ('year','calculatedfinishedsquarefeet'),
                ('year','structuretaxvaluedollarcnt')
    ]

In [48]:
# Categorical pipeline
cat_pipeline = Pipeline([
        ('select_and_dummify', pipes.DF_Selector_GetDummies(cat_dict)),
    ])

# Numerical pipeline
num_pipeline = Pipeline([
        ('selector', pipes.DataFrameSelector(num_atts)),
        ('imputer', Imputer()),
        ('scaler', StandardScaler())
    ])

# interaction pipelines
cat_interact_pipeline = Pipeline([
        ('dummify_and_interact',pipes.Dummify_and_Interact(interact_pairs,cat_dict)),
    ])

num_interact_pipeline = Pipeline([
        ('selector', pipes.DataFrameSelector(num_atts_to_interact)),
        ('imputer', Imputer()),
        ('polynomial_features', PolynomialFeatures(2, include_bias=False)),
        ('scaler', StandardScaler())
    ])

# Full pipeline
feature_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("num_interact_pipeline", num_interact_pipeline),
        ("cat_pipeline", cat_pipeline),
        ("cat_interact_pipeline", cat_interact_pipeline)
    ])

In [49]:
feature_pipeline.fit(data) #fitting the pipeline to the entire properties dataframe

FeatureUnion(n_jobs=1,
       transformer_list=[('num_pipeline', Pipeline(memory=None,
     steps=[('selector', DataFrameSelector(desired_cols=['garagetotalsqft', 'calculatedbathnbr', 'structuretaxvaluedollarcnt', 'bedroomcnt', 'age'])), ('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), (...ty', 'age'), ('year', 'calculatedfinishedsquarefeet'), ('year', 'structuretaxvaluedollarcnt')]))]))],
       transformer_weights=None)

In [50]:
# maindir = "/home/anerdi/Desktop/Zillow"
# properties = pd.read_csv(maindir + "/data/properties_2016_with_2017_tax.csv")

# #proportion of living area
# properties['N-LivingAreaProp'] = properties['calculatedfinishedsquarefeet']/properties['lotsizesquarefeet']

# properties['N-NonLivingAreaProp'] = properties['garagetotalsqft']/properties['lotsizesquarefeet']

# #Ratio of the built structure value to land area
# properties['N-ValueProp'] = properties['structuretaxvaluedollarcnt']/properties['landtaxvaluedollarcnt']

# #Ratio of tax of property over parcel
# properties['N-ValueRatio'] = properties['taxvaluedollarcnt']/properties['taxamount']

# # Pool
# properties['poolsizesum'] = properties['poolsizesum'].fillna(0)
# # properties['Pool'] = (properties['poolsizesum'] > 0).astype(int)
# properties['Pool'] = (properties['pooltypeid2'].fillna(0) + properties['pooltypeid7'].fillna(0)).astype(int)

# properties['regionidcounty'] = properties['regionidcounty'].fillna(9999)
# properties['regionidcity'] = properties['regionidcity'].fillna(9999)
# properties['regionidneighborhood'] = properties['regionidneighborhood'].fillna(9999)
# properties['regionidzip'] = properties['regionidzip'].fillna(9999)
# properties['typeconstructiontypeid'] = properties['typeconstructiontypeid'].fillna(9999)
# properties['airconditioningtypeid'] = properties['airconditioningtypeid'].fillna(9999)
# properties['buildingqualitytypeid'] = properties['buildingqualitytypeid'].fillna(9999)
# properties['heatingorsystemtypeid'] = properties['heatingorsystemtypeid'].fillna(9999)
# properties['propertylandusetypeid'] = properties['propertylandusetypeid'].fillna(9999)
# properties['fips'] = properties['fips'].fillna(9999)

# # some more feature engineering
# properties['age'] = 2017 - properties['yearbuilt']
# properties['additional_rooms_count'] = np.maximum((properties['roomcnt'].values 
#                                                    - properties['calculatedbathnbr'].values
#                                                    - properties['bedroomcnt'].values),0)

# # impute missing num_atts per regionid
# for countyid in properties.regionidcounty.unique():
#     # setup condition
#     cond = properties['regionidcounty'] == countyid
#     indices = np.where(cond)[0]
#     # impute values based on region
#     if countyid != 9999:
#         properties.loc[indices,num_atts] = (properties.loc[indices,num_atts]
#                                 .fillna(properties.loc[indices,num_atts]
#                                 .apply(np.mean)))
#     else:
#         properties.loc[indices,num_atts] = (properties.loc[indices,num_atts]
#                                             .fillna(properties[num_atts]
#                                             .apply(np.mean)))

In [51]:
properties.drop([c for c in properties.columns if c not in num_atts + num_atts_to_interact + cat_atts + ['parcelid']],
               axis=1, inplace=True)

In [52]:
import gc

In [53]:
gc.collect()

215

## Splitting the Training Set

In [37]:
ix_overestimated = np.where(data['logerror'] >= 0)[0]
ix_underestimated = np.where(data['logerror'] < 0)[0]
data_indices = {"over": ix_overestimated, "under": ix_underestimated}

In [38]:
assert ix_overestimated.shape[0] + ix_underestimated.shape[0] == data.shape[0]

## Training Elastic Net

In [17]:
from sklearn.linear_model import ElasticNet, Lars, HuberRegressor
from sklearn.base import clone

import warnings
warnings.filterwarnings("ignore")

In [64]:
models = [
    ("ridge",ElasticNet(alpha=1.25, l1_ratio = 0, max_iter=1000)),
#     ("ridge2",ElasticNet(alpha=10, l1_ratio = 0, max_iter=1000)),
#     ("ridge3",ElasticNet(alpha=15, l1_ratio = 0, max_iter=1000)),
    ("enet", ElasticNet(alpha=0.025, l1_ratio = 0.5, max_iter=1000)),
    ("lasso", ElasticNet(alpha=0.025, l1_ratio = 1, max_iter=1000)),
    ("larm", Lars(n_nonzero_coefs = 1)),
    ("huber", HuberRegressor())
]

In [65]:
test_predictions = pd.read_csv(maindir + "/data/properties_2016_with_2017_tax.csv", usecols=['parcelid'])

for pair in models:
    current_model_name,current_model = pair
    print("Current model: %s" % current_model_name)
    
    for key,val in data_indices.items():
        type_of_zestimate, ix = key, val

        # preprocess current training data
        current_traindata = data.iloc[ix,]

        # get a clone of the model and fit the current training data
        reg = clone(current_model)
        reg.fit(feature_pipeline.transform(current_traindata), current_traindata['logerror'])

#         for month in [10,11,12]:
        for trx_date in ['2016-10-01','2016-11-01','2016-12-01']:
            # obtain predictions on test set
            dateobj = datetime.datetime.strptime(trx_date,'%Y-%m-%d')
            reg_preds = generate_regression_preds(reg, properties,
                  model_name="%s_%d_%s" % (current_model_name,dateobj.month,type_of_zestimate), trx_date=trx_date)
            test_predictions = pd.concat([test_predictions, reg_preds], axis=1)
        
        del reg
        gc.collect()

Current model: ridge
Current model: enet
Current model: lasso
Current model: larm
Current model: huber


In [66]:
test_predictions.head()

,parcelid,ridge_10_under,ridge_11_under,ridge_12_under,ridge_10_over,ridge_11_over,ridge_12_over,enet_10_under,enet_11_under,enet_12_under,...,larm_12_under,larm_10_over,larm_11_over,larm_12_over,huber_10_under,huber_11_under,huber_12_under,huber_10_over,huber_11_over,huber_12_over
0,10754147,-0.080649,-0.080694,-0.080743,0.080019,0.080037,0.080164,-0.070874,-0.070874,-0.070874,...,-0.066404,0.076334,0.076334,0.076334,-0.031762,-0.031762,-0.031762,0.077484,0.077483,0.077483
1,10759547,-0.078461,-0.078506,-0.078556,0.078000,0.078018,0.078145,-0.070874,-0.070874,-0.070874,...,-0.066404,0.076334,0.076334,0.076334,-0.031762,-0.031762,-0.031762,0.077498,0.077498,0.077498
2,10843547,-1.192476,-1.192521,-1.192571,1.802936,1.802954,1.803081,-0.094411,-0.094411,-0.094411,...,-0.066558,0.076430,0.076430,0.076430,-0.121622,-0.121622,-0.121622,0.091934,0.091933,0.091933
3,10859147,-0.115657,-0.115702,-0.115751,0.111204,0.111222,0.111348,-0.098944,-0.098944,-0.098944,...,-0.070872,0.079098,0.079098,0.079098,-0.106854,-0.106854,-0.106854,0.111367,0.111367,0.111367
4,10879947,-0.091642,-0.091688,-0.091737,0.089616,0.089634,0.089760,-0.081486,-0.081486,-0.081486,...,-0.071264,0.079341,0.079341,0.079341,-0.036389,-0.036389,-0.036389,0.093530,0.093530,0.093530


In [67]:
test_predictions.to_csv("/home/anerdi/Desktop/Zillow/twostagemodel/two_stage_preds_linear_models_201617.csv.gz",
                       compression='gzip', index=False)

In [21]:
overestimate_probabilities = pd.read_csv("/home/anerdi/Desktop/Zillow/twostagemodel/overestimate_probs_stacked_ann_rfs_xgbs_lgbms_20162017.csv.gz")
overestimate_probabilities.rename(columns={'stacked_pred':'overestimate_prob'}, inplace=True)

In [22]:
overestimate_probabilities.head()

,parcelid,ann_overestimate_prob,rf2_overestimate_prob,rf3_overestimate_prob,xgb1_overestimate_prob,xgb2_overestimate_prob,lgbm1_overestimate_prob,lgbm2_overestimate_prob,overestimate_prob
0,10754147,0.466125,0.497368,0.560124,0.462231,0.479722,0.511108,0.650032,0.515646
1,10759547,0.412405,0.500739,0.420662,0.525333,0.527149,0.543325,0.646905,0.553857
2,10843547,0.527465,0.636537,0.535855,0.411728,0.522140,0.544722,0.501281,0.466540
3,10859147,0.665014,0.648656,0.456073,0.580416,0.570832,0.590595,0.576605,0.581211
4,10879947,0.477125,0.524182,0.483156,0.514524,0.509930,0.515008,0.404796,0.483875


#### Merging Results

In [68]:
test_predictions = pd.merge(test_predictions, overestimate_probabilities, on='parcelid')

In [69]:
for pair in models:
    current_model_name, current_model = pair
    # combine over and under to get prediction
    for month in [10,11,12]:
        test_predictions['%s_%d' % (current_model_name, month)] = (
                test_predictions['%s_%d_over' % (current_model_name, month)]*test_predictions['overestimate_prob'] 
                + test_predictions['%s_%d_under' % (current_model_name, month)]*(1 - test_predictions['overestimate_prob']))

In [70]:
test_predictions.head()

,parcelid,ridge_10_under,ridge_11_under,ridge_12_under,ridge_10_over,ridge_11_over,ridge_12_over,enet_10_under,enet_11_under,enet_12_under,...,enet_12,lasso_10,lasso_11,lasso_12,larm_10,larm_11,larm_12,huber_10,huber_11,huber_12
0,10754147,-0.080649,-0.080694,-0.080743,0.080019,0.080037,0.080164,-0.070874,-0.070874,-0.070874,...,0.005381,0.005321,0.005321,0.005321,0.007198,0.007198,0.007198,0.024570,0.024570,0.024570
1,10759547,-0.078461,-0.078506,-0.078556,0.078000,0.078018,0.078145,-0.070874,-0.070874,-0.070874,...,0.011032,0.010983,0.010983,0.010983,0.012653,0.012653,0.012653,0.028753,0.028752,0.028752
2,10843547,-1.192476,-1.192521,-1.192571,1.802936,1.802954,1.803081,-0.094411,-0.094411,-0.094411,...,-0.004626,-0.004611,-0.004611,-0.004611,0.000151,0.000151,0.000151,-0.021990,-0.021990,-0.021990
3,10859147,-0.115657,-0.115702,-0.115751,0.111204,0.111222,0.111348,-0.098944,-0.098944,-0.098944,...,0.018371,0.017932,0.017932,0.017932,0.016292,0.016292,0.016292,0.019979,0.019978,0.019978
4,10879947,-0.091642,-0.091688,-0.091737,0.089616,0.089634,0.089760,-0.081486,-0.081486,-0.081486,...,0.000234,0.000190,0.000190,0.000190,0.001610,0.001610,0.001610,0.026476,0.026475,0.026476


In [71]:
model_name = 'lasso' # can be any one of ridge, enet, lasso, huber, or larm
new_submission = DataFrame({'ParcelId': test_predictions['parcelid'],
                           '201610':test_predictions['%s_10' % model_name],
                           '201611':test_predictions['%s_11' % model_name],
                           '201612':test_predictions['%s_12' % model_name],
})
new_submission['201710'] = 0
new_submission['201711'] = 0
new_submission['201712'] = 0

In [72]:
new_submission.head()

,201610,201611,201612,ParcelId,201710,201711,201712
0,0.005321,0.005321,0.005321,10754147,0,0,0
1,0.010983,0.010983,0.010983,10759547,0,0,0
2,-0.004611,-0.004611,-0.004611,10843547,0,0,0
3,0.017932,0.017932,0.017932,10859147,0,0,0
4,0.000190,0.000190,0.000190,10879947,0,0,0


In [73]:
mean_absolute_errors(new_submission.round(4), data)

(0.061798573437813926,
 0.06082667031763426,
 0.07336158711903397,
 0.063944837274642952)

In [61]:
# alpha = 1.25
mean_absolute_errors(new_submission.round(4), data)

(0.06179845288326326,
 0.060861117196056934,
 0.0733271420356527,
 0.063945118239288376)

In [28]:
# alpha = 1.25
mean_absolute_errors(new_submission.round(4), data)

(0.06179774964838238,
 0.06086330777656092,
 0.07332208165612432,
 0.063944146569889893)

In [62]:
new_submission.round(4).to_csv("/home/anerdi/Desktop/Zillow/submissions/two_stage_stage1_annrfsxgbslgbms_stage2_ridge_201617_year.csv.gz", index=False,
                     compression='gzip')